# MYMail_review – Notas / comandos rápidos

Este notebook sirve como recordatorio de comandos para:
- Subir nuevos ficheros a **entrada** (CosmosDB)
- Gestión de **usuarios** (CosmosDB)
- Despliegue de imagen (ACR) + restart App Service


## 0) CosmosDB (setup + migración)

Variables mínimas en `.env` / App Service Configuration:
- `STORAGE_BACKEND=cosmos`
- `COSMOS_ENDPOINT=<URI>`
- `COSMOS_KEY=<KEY>`
- `COSMOS_DATABASE=mymailreview`

Crear DB/containers (una vez):


Test de conexión a CosmosDB:


In [ ]:
!python Testing/test_cosmos.py


## 1) Subir un nuevo Excel a ENTRADA (CosmosDB)

Carga el Excel y crea registros en el contenedor `entrada` de CosmosDB.

Nota: si un `record_json` supera el límite seguro, se guarda el payload en Blob (legacy) y en Cosmos queda una referencia.


In [ ]:
!python scripts/load_entrada.py --excel "C:\\ruta\\Validados_V3.xlsx" --sheet "1 dic - 8 dic"

In [ ]:
!python scripts/load_entrada.py --excel "Validados_V3.xlsx" --sheet "1 dic - 8 dic" --partition "default"

In [ ]:
!python scripts/load_entrada.py --excel "Validados_V3.xlsx" --sheet "1 dic - 8 dic" --replace

Flags útiles:
- `--replace` borra la partición antes de cargar
- `--partition` PartitionKey destino (por defecto: `default`)


## 2) Gestión de usuarios (CosmosDB)

### 2.1 Crear/actualizar usuario


In [ ]:
!python scripts/manage_users.py add --username "usuario1" --password "MiPassword" --role revisor

In [ ]:
!python scripts/manage_users.py add --username "admin" --password "MiPassword" --role admin

### 2.2 Cambiar rol


In [ ]:
!python scripts/manage_users.py set-role --username "usuario1" --role admin

### 2.3 Actualizar contraseña


In [ ]:
!python scripts/manage_users.py set-password --username "usuario1" --password "NuevaPassword"

### 2.4 Listar usuarios


In [ ]:
!python scripts/manage_users.py list

## Requisitos

- Variables en `.env` / `config.py` (CosmosDB: `STORAGE_BACKEND=cosmos`, `COSMOS_ENDPOINT`, `COSMOS_KEY`).
- `AZURE_STORAGE_CONNECTION_STRING` solo si usas blobs (`revisiones` / payloads grandes de `entrada`).
- Dependencias instaladas: `pip install -r requirements.txt`


## 3) Test de Azure OpenAI (legacy)

Usa el script `Testing/test_azure_openai.py` para validar que el endpoint + deployment + api-version funcionan.


In [ ]:
! .\\.venv\\Scripts\\python.exe Testing/test_azure_openai.py --prompt "Responde exactamente: OK" --temperature 1 --max-tokens 256

## 4) Desplegar en cloud (ACR)

Comandos para construir, etiquetar y publicar la imagen en Azure Container Registry.

Antes de empezar:
- Asegúrate de que **Docker Desktop** está iniciado (si no, `docker build` fallará).
- Asegúrate de estar logueado en **Azure**.

Configurar App Service con CosmosDB (placeholders):


In [ ]:
!az webapp config appsettings set --resource-group <RG> --name <APP_NAME> --settings STORAGE_BACKEND=cosmos COSMOS_ENDPOINT=<URI> COSMOS_KEY=<KEY> COSMOS_DATABASE=mymailreview


In [ ]:
!docker info


In [ ]:
!az login


In [ ]:
!az acr login -n mymailrevieweuweconreg002


In [ ]:
!docker build -t mymailreview .
!docker tag mymailreview mymailrevieweuweconreg002.azurecr.io/mymailreview:latest
!docker push mymailrevieweuweconreg002.azurecr.io/mymailreview:latest


Reiniciar App Service para que cargue el nuevo contenedor (ajusta `<RG>` y `<APP_NAME>`):


In [ ]:
!az webapp restart --resource-group <RG> --name <APP_NAME>


## OLD (legacy) – Blob / Azure Table Storage

La app antigua usaba Azure **Table Storage** para persistencia y Azure **Blob** para entrada/revisiones.

Variables legacy:
- `STORAGE_BACKEND=tables`
- `AZURE_STORAGE_CONNECTION_STRING=...`
- `TABLE_USERS/TABLE_LOGS/TABLE_RESULTADOS/TABLE_DESCARTES/TABLE_ENTRADA`
- `AZURE_BLOB_CONTAINER_ENTRADA=entrada`
- `AZURE_BLOB_CONTAINER_REVISIONES=revisiones`

Notas:
- Aunque uses CosmosDB, el Blob se puede seguir usando para `revisiones` y para payloads grandes de `entrada`.
